In [1]:
import os
import src.data_manager as data_manager
import src.gfdl as gfdl
import src.util as util
import src.datelabel as datelabel
import src.shared_diagnostic as shared_diagnostic
import yaml

In [2]:
default_args = util.read_yaml('./src/config.yml')
cmdline_args = {'CODE_ROOT':os.getcwd()}
test_dir = '/archive/oar.gfdl.cmip6/CM4/warsaw_201710_om4_v1.0.1/CM4_historical/gfdl.ncrc4-intel16-prod-openmp/pp'
test_case = {
    'CASENAME': 'CM4_historical', 'model': 'CM4', 
     'variable_convention': 'CMIP',
    'FIRSTYR': 1970, 'LASTYR': 1989,
    'pod_list': []
}
test_varlist = [
    {'var_name':'rlut', 'freq':'day'},
    {'var_name':'pr', 'freq':'day'},
    {'var_name':'omega500', 'freq':'day'},
    {'var_name':'u200', 'freq':'day'},
    {'var_name':'u850', 'freq':'day'}
]

In [3]:
config = util.parse_mdtf_args(None, cmdline_args, default_args)
util.PathManager(config['paths'])
dm = gfdl.GfdlppDataManager(test_dir, test_case)
pod = shared_diagnostic.Diagnostic('Wheeler_Kiladis')

In [4]:
dm.pods = [pod]
for pod in dm.pods:
    dm._setup_pod(pod)

In [6]:
ds = dm.pods[0].varlist[0].copy()
print ds

DataSet({'CF_name': 'rlut_var', 'requirement': 'required', 'name': 'rlut_var', 'date_freq': DateFrequency('1da'), 'required': True, 'date_range': DateRange('1970-1989'), 'remote_resource': None, 'var_name': 'rlut_var', 'name_in_model': 'rlut', 'units': None, 'freq': 'day', 'local_resource': '/local2/home/MDTF/inputdata/model/CM4_historical/day/CM4_historical.rlut.day.nc', 'alternates': []})


In [7]:
dm.query_dataset(ds)
print ds

DataSet({'CF_name': 'rlut_var', 'requirement': 'required', 'name': 'rlut_var', 'date_freq': DateFrequency('1da'), 'required': True, 'date_range': DateRange('1970-1989'), 'remote_resource': [DataSet({'end_date': '19741231', 'date_freq': DateFrequency('1da'), 'component': 'atmos_cmip_2deg_daily_2D', 'date_range': DateRange('19700101-19741231'), 'remote_resource': 'atmos_cmip_2deg_daily_2D/ts/daily/5yr/atmos_cmip_2deg_daily_2D.19700101-19741231.rlut.nc', 'file': 'atmos_cmip_2deg_daily_2D.19700101-19741231.rlut.nc', 'field_name': 'rlut', 'local_resource': None, 'name': None, 'chunk_freq': DateFrequency('5yr'), 'component2': 'atmos_cmip_2deg_daily_2D', 'units': None, 'start_date': '19700101', 'dir': 'atmos_cmip_2deg_daily_2D/ts/daily/5yr'}), DataSet({'end_date': '19791231', 'date_freq': DateFrequency('1da'), 'component': 'atmos_cmip_2deg_daily_2D', 'date_range': DateRange('19750101-19791231'), 'remote_resource': 'atmos_cmip_2deg_daily_2D/ts/daily/5yr/atmos_cmip_2deg_daily_2D.19750101-197912

In [8]:
util.find_files(test_dir, '*/ts/daily/*.rlut.nc')

['atmos_cmip/ts/daily/5yr/atmos_cmip.18550101-18591231.rlut.nc',
 'atmos_cmip/ts/daily/5yr/atmos_cmip.18500101-18541231.rlut.nc',
 'atmos_cmip/ts/daily/5yr/atmos_cmip.18600101-18641231.rlut.nc',
 'atmos_cmip/ts/daily/5yr/atmos_cmip.18650101-18691231.rlut.nc',
 'atmos_cmip/ts/daily/5yr/atmos_cmip.18700101-18741231.rlut.nc',
 'atmos_cmip/ts/daily/5yr/atmos_cmip.18750101-18791231.rlut.nc',
 'atmos_cmip/ts/daily/5yr/atmos_cmip.18800101-18841231.rlut.nc',
 'atmos_cmip/ts/daily/5yr/atmos_cmip.18850101-18891231.rlut.nc',
 'atmos_cmip/ts/daily/5yr/atmos_cmip.18900101-18941231.rlut.nc',
 'atmos_cmip/ts/daily/5yr/atmos_cmip.18950101-18991231.rlut.nc',
 'atmos_cmip/ts/daily/5yr/atmos_cmip.19000101-19041231.rlut.nc',
 'atmos_cmip/ts/daily/5yr/atmos_cmip.19050101-19091231.rlut.nc',
 'atmos_cmip/ts/daily/5yr/atmos_cmip.19100101-19141231.rlut.nc',
 'atmos_cmip/ts/daily/5yr/atmos_cmip.19150101-19191231.rlut.nc',
 'atmos_cmip/ts/daily/5yr/atmos_cmip.19200101-19241231.rlut.nc',
 'atmos_cmip/ts/daily/5yr

In [11]:
for f in ds.remote_resource:
    print f.remote_resource

atmos_cmip_2deg_daily_2D/ts/daily/5yr/atmos_cmip_2deg_daily_2D.19700101-19741231.rlut.nc
atmos_cmip_2deg_daily_2D/ts/daily/5yr/atmos_cmip_2deg_daily_2D.19750101-19791231.rlut.nc
atmos_cmip_2deg_daily_2D/ts/daily/5yr/atmos_cmip_2deg_daily_2D.19800101-19841231.rlut.nc
atmos_cmip_2deg_daily_2D/ts/daily/5yr/atmos_cmip_2deg_daily_2D.19850101-19891231.rlut.nc
atmos_cmip/ts/daily/5yr/atmos_cmip.19700101-19741231.rlut.nc
atmos_cmip/ts/daily/5yr/atmos_cmip.19750101-19791231.rlut.nc
atmos_cmip/ts/daily/5yr/atmos_cmip.19800101-19841231.rlut.nc
atmos_cmip/ts/daily/5yr/atmos_cmip.19850101-19891231.rlut.nc


In [12]:
print len(ds.remote_resource)
print ds.remote_resource[0]

8
DataSet({'end_date': '19741231', 'date_freq': DateFrequency('1da'), 'component': 'atmos_cmip_2deg_daily_2D', 'date_range': DateRange('19700101-19741231'), 'remote_resource': 'atmos_cmip_2deg_daily_2D/ts/daily/5yr/atmos_cmip_2deg_daily_2D.19700101-19741231.rlut.nc', 'file': 'atmos_cmip_2deg_daily_2D.19700101-19741231.rlut.nc', 'field_name': 'rlut', 'local_resource': None, 'name': None, 'chunk_freq': DateFrequency('5yr'), 'component2': 'atmos_cmip_2deg_daily_2D', 'units': None, 'start_date': '19700101', 'dir': 'atmos_cmip_2deg_daily_2D/ts/daily/5yr'})


In [13]:
cmpts = dm._select_model_component([ds])
print cmpts

['atmos_cmip']


In [14]:
files_to_fetch = []
ds_list = [d for d in ds.remote_resource if (d.component in cmpts)]
# take longest chunk frequency (revisit?)
chunk_freq = max({d.chunk_freq for d in ds_list})
ds.remote_resource = [d for d in ds_list if (d.chunk_freq == chunk_freq)]
assert ds.remote_resource # shouldn't have eliminated everything
files_to_fetch.extend(ds.remote_resource)
print files_to_fetch
for f in files_to_fetch:
    f.local_resource = dm.local_path(f)
foo2 = [f for f in set(files_to_fetch) if not dm.local_data_is_current(f)]
print foo2

[DataSet({'end_date': '19741231', 'date_freq': DateFrequency('1da'), 'component': 'atmos_cmip', 'date_range': DateRange('19700101-19741231'), 'remote_resource': 'atmos_cmip/ts/daily/5yr/atmos_cmip.19700101-19741231.rlut.nc', 'file': 'atmos_cmip.19700101-19741231.rlut.nc', 'field_name': 'rlut', 'local_resource': None, 'name': None, 'chunk_freq': DateFrequency('5yr'), 'component2': 'atmos_cmip', 'units': None, 'start_date': '19700101', 'dir': 'atmos_cmip/ts/daily/5yr'}), DataSet({'end_date': '19791231', 'date_freq': DateFrequency('1da'), 'component': 'atmos_cmip', 'date_range': DateRange('19750101-19791231'), 'remote_resource': 'atmos_cmip/ts/daily/5yr/atmos_cmip.19750101-19791231.rlut.nc', 'file': 'atmos_cmip.19750101-19791231.rlut.nc', 'field_name': 'rlut', 'local_resource': None, 'name': None, 'chunk_freq': DateFrequency('5yr'), 'component2': 'atmos_cmip', 'units': None, 'start_date': '19750101', 'dir': 'atmos_cmip/ts/daily/5yr'}), DataSet({'end_date': '19841231', 'date_freq': DateFre

AttributeError: name_in_model

In [ ]:
for f in files_to_fetch:
    print f.remote_resource

In [ ]:
import tempfile
import os

In [ ]:
tempfile.tempdir = os.getcwd()
foo1 = tempfile.mkdtemp(prefix='MDTF_temp_')
print foo1
os.rmdir(foo1)

In [ ]:
print tempfile.tempdir

In [ ]:
print ds.__hash__()
print hex(ds.__hash__())

In [15]:
print os.listdir(test_dir)

['.dec', '.checkpoint', 'atmos_cmip', 'atmos_cmip_2deg_daily_2D', 'atmos_cmip_2deg_daily_3D', 'atmos', 'atmos_level_cmip', 'atmos_level_cmip_2deg_daily', 'atmos_level', 'atmos_month_aer', 'atmos_diurnal_cmip', 'atmos_diurnal', 'atmos_scalar', 'tracer_level', 'aerosol_cmip', 'aerosol_cmip_2deg', 'land', 'land_cubic', 'land_instant_cubic', 'land_cmip', 'land_cmip_2deg', 'river', 'river_cubic', 'river_instant_cubic', 'land_month_past', 'land_month_inst_past', 'land_month_crop', 'land_month_inst_crop', 'land_month_ntrl', 'land_month_inst_ntrl', 'land_month_scnd', 'land_month_inst_scnd', 'LUmip_month_psl', 'LUmip_month_crp', 'LUmip_month_pst', 'LUmip_year_psl', 'LUmip_year_crp', 'LUmip_year_pst', 'ocean_daily_cmip', 'ocean_daily_gfdl', 'ocean_daily_1x1deg', 'ocean_monthly', 'ocean_monthly_1x1deg', 'ocean_month_rho2', 'ocean_month_rho2_1x1deg', 'ocean_monthly_z', 'ocean_monthly_z_1x1deg', 'ocean_annual', 'ocean_annual_1x1deg', 'ocean_annual_rho2', 'ocean_annual_rho2_1x1deg', 'ocean_annual_z'

In [25]:
def test_find(root_dir, dataset):
    cmpts = [d for d in os.listdir(root_dir) if not d.startswith('.')]
    candidates = []
    query = '*.{}.nc'.format(dataset.name_in_model)
    for rp1 in cmpts:
        rp2 = os.path.join(rp1, 'ts', dataset.date_freq.format_frepp())
        ap2 = os.path.join(root_dir, rp2)
        if os.path.exists(ap2):
            chunk_freqs = [d for d in os.listdir(ap2) if not d.startswith('.')]
            for rp3 in chunk_freqs:
                ap3 = os.path.join(ap2, rp3)
                print ap3
                paths = util.run_command([
                    'find', ap3, '-name', query, '-print','-quit'
                ])
                if paths:
                    print 'GOOD'
                    candidates.append(ap3)
    return candidates

In [26]:
test_find(test_dir, ds)

/archive/oar.gfdl.cmip6/CM4/warsaw_201710_om4_v1.0.1/CM4_historical/gfdl.ncrc4-intel16-prod-openmp/pp/atmos_cmip/ts/daily/5yr
GOOD
/archive/oar.gfdl.cmip6/CM4/warsaw_201710_om4_v1.0.1/CM4_historical/gfdl.ncrc4-intel16-prod-openmp/pp/atmos_cmip/ts/daily/35yr
GOOD
/archive/oar.gfdl.cmip6/CM4/warsaw_201710_om4_v1.0.1/CM4_historical/gfdl.ncrc4-intel16-prod-openmp/pp/atmos_cmip_2deg_daily_2D/ts/daily/5yr
GOOD
/archive/oar.gfdl.cmip6/CM4/warsaw_201710_om4_v1.0.1/CM4_historical/gfdl.ncrc4-intel16-prod-openmp/pp/atmos_cmip_2deg_daily_2D/ts/daily/35yr
GOOD
/archive/oar.gfdl.cmip6/CM4/warsaw_201710_om4_v1.0.1/CM4_historical/gfdl.ncrc4-intel16-prod-openmp/pp/atmos_cmip_2deg_daily_3D/ts/daily/5yr
/archive/oar.gfdl.cmip6/CM4/warsaw_201710_om4_v1.0.1/CM4_historical/gfdl.ncrc4-intel16-prod-openmp/pp/atmos_cmip_2deg_daily_3D/ts/daily/35yr
/archive/oar.gfdl.cmip6/CM4/warsaw_201710_om4_v1.0.1/CM4_historical/gfdl.ncrc4-intel16-prod-openmp/pp/atmos_level_cmip_2deg_daily/ts/daily/5yr
/archive/oar.gfdl.cmip

['/archive/oar.gfdl.cmip6/CM4/warsaw_201710_om4_v1.0.1/CM4_historical/gfdl.ncrc4-intel16-prod-openmp/pp/atmos_cmip/ts/daily/5yr',
 '/archive/oar.gfdl.cmip6/CM4/warsaw_201710_om4_v1.0.1/CM4_historical/gfdl.ncrc4-intel16-prod-openmp/pp/atmos_cmip/ts/daily/35yr',
 '/archive/oar.gfdl.cmip6/CM4/warsaw_201710_om4_v1.0.1/CM4_historical/gfdl.ncrc4-intel16-prod-openmp/pp/atmos_cmip_2deg_daily_2D/ts/daily/5yr',
 '/archive/oar.gfdl.cmip6/CM4/warsaw_201710_om4_v1.0.1/CM4_historical/gfdl.ncrc4-intel16-prod-openmp/pp/atmos_cmip_2deg_daily_2D/ts/daily/35yr']

In [27]:
os.listdir('/archive/oar.gfdl.cmip6/CM4/warsaw_201710_om4_v1.0.1/CM4_historical/gfdl.ncrc4-intel16-prod-openmp/pp/atmos_cmip_2deg_daily_2D/ts/daily/35yr')


['atmos_cmip_2deg_daily_2D.19800101-20141231.rlus.nc',
 'atmos_cmip_2deg_daily_2D.19800101-20141231.albisccp.nc',
 'atmos_cmip_2deg_daily_2D.19800101-20141231.wap500.nc',
 'atmos_cmip_2deg_daily_2D.19800101-20141231.ccb.nc',
 'atmos_cmip_2deg_daily_2D.19800101-20141231.ta700.nc',
 'atmos_cmip_2deg_daily_2D.19800101-20141231.cct.nc',
 'atmos_cmip_2deg_daily_2D.19800101-20141231.rlutcs.nc',
 'atmos_cmip_2deg_daily_2D.19800101-20141231.cldnvi.nc',
 'atmos_cmip_2deg_daily_2D.19800101-20141231.rlut.nc',
 'atmos_cmip_2deg_daily_2D.19800101-20141231.clhcalipso.nc',
 'atmos_cmip_2deg_daily_2D.19800101-20141231.rsdscsdiff.nc',
 'atmos_cmip_2deg_daily_2D.19800101-20141231.clivi.nc',
 'atmos_cmip_2deg_daily_2D.19800101-20141231.rsdscs.nc',
 'atmos_cmip_2deg_daily_2D.19800101-20141231.cllcalipso.nc',
 'atmos_cmip_2deg_daily_2D.19800101-20141231.rsdsdiff.nc',
 'atmos_cmip_2deg_daily_2D.19800101-20141231.clmcalipso.nc',
 'atmos_cmip_2deg_daily_2D.19800101-20141231.rsds.nc',
 'atmos_cmip_2deg_daily_2

In [35]:
    def query_dataset2(dataset, root_dir):
        """Return set of candidate directories for data files.
        """
        if 'component' in dataset:
            cmpts = [dataset.component]
        else:
            cmpts = [d for d in os.listdir(root_dir) if not d.startswith('.')]
        candidate_dirs = []
        suffix_query = '.{}.nc'.format(dataset.name_in_model)
        for component in cmpts:
            subdir_rel = os.path.join(component, 'ts', dataset.date_freq.format_frepp())
            subdir_abs = os.path.join(root_dir, subdir_rel)
            if not os.path.exists(subdir_abs):
                continue
            chunk_freqs = [d for d in os.listdir(subdir_abs) if not d.startswith('.')]
            for freq in chunk_freqs:
                paths = util.run_command([
                    'find', os.path.join(subdir_abs, freq), '-name', '*'+suffix_query, '-print', '-quit'
                ])
                if paths:
                    candidate_dirs.append(os.path.join(subdir_rel, freq))
        if not candidate_dirs:
            raise DataQueryFailure(dataset, 'No files found in {}'.format(root_dir))

        for d in candidate_dirs:
            files = [dm.parse_pp_path(f) for f in os.listdir(os.path.join(root_dir,d)) \
                if f.endswith(suffix_query)]
            try:
                remote_range = datelabel.DateRange([ds.date_range for ds in files])
            except ValueError:
                # Something's messed up with remote files if we get here
                # should probably log an error
                continue
            if remote_range.contains(dataset.date_range):
                dataset.remote_resource.extend(
                    [ds for ds in files if (ds.date_range in dataset.date_range)]
                )
        if not dataset.remote_resource:
            raise DataQueryFailure(dataset, 
                "Couldn't cover date range {} with files in {}".format(
                    dataset.date_range, root_dir))

In [36]:
query_dataset2(ds, test_dir)

ValueError: 